# Ich baue mir einen Mastodon-Bot mit ein paar Zeilen Code und Autdone

[Autodone](https://autodone.idh.uni-koeln.de) ist ein Tool, das auf Basis einer importierten Tabelle Posts auf Mastodon veröffentlicht.
Eine Anleitung findet sich [hier](https://autodone.idh.uni-koeln.de/usage).

Der folgende Code zeigt, wie über die MD-Api Daten gezogen und in ein autodon-kompatibles Format gebracht werden können.
Ausgangspunkt ist eine Suche über das MD-Interface ([URL der Suche](https://berlin.museum-digital.de/objects?s=institution%3A29+metadata_rights_images%3A%3DCC_BY-SA&section=results_list&mode=grid&startwert=48&gbreitenat=100).
Gerade die rechtliche Situation ist besonders zu beachten, um nicht versehentlich Objekte urheberrechtswidrig zur verwenden.

Die Posts werden im folgenden so terminiert, dass der Bot morgen um 12:00 den ersten Beitrag veröffentlicht und dann jeden Tag einen neuen.
Die Posts werden jeweils mit einem Vorschaubild versehen.

Die Tabelle wird, falls das Notebook lokal läuft, in einer CSV-Datei gespeichert; falls das Notebook in der JupyterLite-Umgebung läuft, wird das Ergebnis einfach im Browser ausgegeben. 

In [ ]:
import requests

In [ ]:
from datetime import datetime, timedelta

def generate_dates(days):
    '''Funktion die eine Liste von <days> aufeinanderfolgenden Datumsangaben
    erzeugt.'''
    today = datetime.now() + timedelta(days = 1)
    return [(today + timedelta(days=i)).strftime("%d.%m.%Y") for i in range(days)]

In [ ]:
# über die Suchmaske der Weboberfläche konfigurierte Anfrage
# JSON erreichbar mit Strg + Rechtsklick => Exportieren
# Abgerufen wurden Objekte des Industriesalons Schöneweide mit
# geeigneter Lizenz (CC BY-SA)
def getMDdata(offset, rows):
    apiCall = f"https://berlin.museum-digital.de/json/objects?&s=collection%3A1120&startwert={offset}&gbreitenat={rows}"
    return requests.get(apiCall).json()

def objectMetadata(objectID):
    url = f"https://berlin.museum-digital.de/json/object/{objectID}"
    res = requests.get(url)
    return res.json()

def createPostContent(objectDict:dict, objectID):
    objName = objectDict.get('object_name').partition('. ')[0]
    objDesc = objectDict.get('object_description').partition('. ')[0]
    mainImg = [ _ for _ in objectDict['object_images'] if _['is_main'] == "j"][0]
    mainImgPreview = "https://asset.museum-digital.org/berlin/" + mainImg['folder'] + "/" + mainImg['filename_loc']
    copyrightLine = f"© {mainImg['creator']} und {mainImg['owner']}, {mainImg['rights']}"
    hashtags = "#berlin #photography #industriesalon"
    return f"""{objName}.
    
    {objDesc}
    
    {copyrightLine}
    
    Erfahren Sie mehr über das Objekt auf #museumdigital unter https://berlin.museum-digital.de/json/object/{objectID}
    
    {hashtags}""", mainImgPreview

In [ ]:
total = getMDdata(0,1)[0].get('total')

In [ ]:
ID = "112684"
pC = createPostContent(objectMetadata(ID), ID)
print(pC[0])
print(pC[1])

In [ ]:
rows = 100
autodone_table = list()
dates = generate_dates(total)

autodone_table.append(["Date", "Time", "Text", "Picture", "Pic description"])

for i in range(0,total + 1, rows):
    res = getMDdata(i, rows)
    for r in res:
        autodone_table.append([
            dates.pop(0), "10:00",
            f"{r.get('objekt_name')} (Link: https://berlin.museum-digital.de/object/{r.get('objekt_id')})",
            r.get('image').replace('data', 'https://asset.museum-digital.org//', 1),
            ""
        ])

In [ ]:
import csv
try:
    ## Bei lokalem Betrieb kann Tabelle als csv gespeichert werden
    OUT = open("autodone_table_plewka.csv", "w")
    writer = csv.writer(OUT, delimiter='\t')
    writer.writerows(autodone_table)
    OUT.close()
    ## Wird der Code in der JupyterLite-Umgebung ausgeführt, kann
    ## lokal nichts gespeichert werden. Daher wird die Liste einfach
    ## nur unter der Zelle ausgegeben.
except OSError:
    for row in autodone_table:
        print("\t".join(row))